决策树算法实现
完成了基础信息熵，c4.5，CART三种决策树算法+剪枝的实现
希望输入的形式是全数字的D和特征数量n的[0,1,2,3....,n]的A

模型比对实验结果
<img src="image/4/compare_result.jpg" alt="alt text" style="width: 50%; height: auto;">

神了！见图
<img src="image/4/one_error_about_para_position.png" alt="alt text" style="width: 50%; height: auto;">

In [2]:
import math 
class Node(object):
  # def __init__(self,node_D,feature_index=None,parent=None,children=[],leaf_value=None):
  
    def __init__(self, node_D, feature_val=None, feature_index=None, parent=None, children=None, leaf_value=None):  
        self.node_D = node_D #节点数据  
        self.feature_val=feature_val #属性值
        self.feature_index = feature_index #属性索引 
        self.parent = parent  #父节点
        self.children = children if children is not None else [] # 子节点们  
        self.leaf_value = leaf_value #叶子节点的值（好瓜坏瓜）


class TreeGenerate(object):
  def __init__(self,D,A):

     self.D=D #数据集
     self.A=A #属性集
     self.root=None
     self._build_tree(D,A)

    # calculate_pk([[1,1],[2,2]])
    # {'1': 0.5, '2': 0.5}
  def calculate_pk(self,D):
    A=[]
    for i in range(len(D)): 
      # 提取最后一列数据
      A.append(D[i][-1])
        
    # 创建一个字典来存储每个值（转换为字符串后）出现的次数  
    count_dict = {}     
    # 统计每个值（转换为字符串后）出现的次数  
    for item in A:  
        # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
        # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
        item_str = str(item)  
        if item_str in count_dict:  
            count_dict[item_str] += 1  
        else:  
            count_dict[item_str] = 1       
    # 计算列表中元素的总数  
    total_count = len(A)         
    # 计算每个值（转换为字符串后）出现的次数占总次数的比例  
    proportion_dict = {item_str: count / total_count for item_str, count in count_dict.items()}     
    # 打印结果  
    return proportion_dict

  # print(calculate_Entropy(['是','是','是','是','是','是','是','是','否','否','否','否','否','否','否','否','否']))
  # 0.9975025463691153
  def calculate_Entropy(self,D):

    p = self.calculate_pk(D)
    entropy = 0
    for i in p.values():
      entropy += i*math.log(i,2)
    return -entropy
  # 计算第二个（索引1）属性的信息增益
  # calculate_Gain(0,[[1,2,3,0],[2,3,1,0],[3,1,2,0],[1,2,3,0],[2,3,1,0],[3,1,2,1]],1)
  # 0.3166890883150208 ['2', '3', '1']
  # [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

  def calculate_Gain(self,D,a):
      if D==[]:
        print('D is empty')
        return 0
      if a<0 or a>=len(D[0]):
        print('a is not in D')
        return 0
      
      Ent_D = self.calculate_Entropy(D)
      # print(Ent_D)
      Ent_Di={}
      count_dict = {}  # 记录指定属性的取值的次数
      attribute_name=[]   # 记录指定属性的取值
      
      # 统计每个值（转换为字符串后）出现的次数  
      for item in D:  
      # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
      # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
        item_str = str(item[a])  
        if item_str in count_dict:  
          count_dict[item_str] += 1  
          Ent_Di[item_str].append(item)
        else:  
          count_dict[item_str] = 1
          Ent_Di[item_str]=[]
          Ent_Di[item_str].append(item)
          attribute_name.append(item_str)       

      # 计算列表中元素的总数
      total_count = len(D)
      # 计算每个值（转换为字符串后）出现的次数占总次数的比例
      attribute_weight = [count / total_count for count in count_dict.values()]
      # print(attribute_weight)
      # print(attribute_name)
      gain=Ent_D
      for i in range(len(attribute_name)):
        gain-=attribute_weight[i]*self.calculate_Entropy(Ent_Di[attribute_name[i]])

      return gain,attribute_name

  # 选择最大信息增益属性
  # choose_A(0,[[2,2,3,0],[2,2,1,0],[2,2,2,0],[1,2,3,0],[3,3,3,0],[3,1,2,1]],[0,1,2])
  # (1, ['2', '3', '1'])
  def choose_A(self,D,A):
        max_gain=0
        for i in range(len(A)):
          gain,attribute_name=self.calculate_Gain(D,i)
          if gain>max_gain:
            max_gain=gain
            max_attribute_index=i
            max_attribute_name=attribute_name
        print(max_attribute_index,max_gain)
        return max_attribute_index,max_attribute_name
  # def __init__(self,node_D,feature_index,parent=None,childs=[],leaf_value=None):
  # self.node_D=node_D #节点数据
  # self.feature_index = feature_index #属性索引
  # self.parent = parent #父节点
  # self.childs = childs #子节点们
  # self.leaf_value = leaf_value  #叶子节点的值（好瓜坏瓜）
  # 递归构建决策树  
  def _build_tree(self, D, A, parent=None, now_feature_val=None):  
      # 如果所有样本都属于同一类，则返回叶子节点  
      if len(set([item[-1] for item in D])) == 1:  
          leaf_value = D[0][-1]  
          print('1',leaf_value)
          node=Node(D, feature_val=now_feature_val,leaf_value=leaf_value)  
          node.parent=parent
          parent.children.append(node)
          print('这是个叶子节点',f'他的父节点是{node.parent}他的节点D是{node.node_D}他的叶子节点值是:{node.leaf_value}')

          return node
      # 如果没有更多特征可以选择，则返回叶子节点（多数类投票）  
      if not A:  
          leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)  
          print('2',leaf_value)
          node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
          node.parent=parent
          parent.children.append(node)
          print('这是个叶子节点',f'他的父节点是{node.parent}他的节点D是{node.node_D}他的叶子节点值是:{node.leaf_value}')
          return node  
      if D==[]:
        print('D is empty')
        leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)
        node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
        node.parent=parent
        parent.children.append(node)
        print('这是个叶子节点',f'他的父节点是{node.parent}他的节点D是{node.node_D}他的叶子节点值是:{node.leaf_value}')
        return node
       

      # 选择最佳特征  
      feature_index, feature_name = self.choose_A(D, A)  
      print( feature_index,feature_name)
      # 创建根节点 
      if self.root==None: 
        print('创建根节点')
        root = Node(D, feature_val=self.A[feature_index],feature_index=feature_index)
        self.root=root
        new_parent=root
        node=root
      else:
        print('创建中间节点')
        node=Node(D, feature_val=now_feature_val,feature_index=feature_index)
        parent.children.append(node)
        node.parent=parent
        new_parent=node

      # 递归地为每个特征值创建子节点  
      for name in feature_name:  
          print('name',name)
          sub_D = [item for item in D if item[feature_index] == name]  
          sub_A = [i for i in A if i != feature_index]  # 移除已使用的特征  
          print('sub_D',sub_D)
          print('sub_A',sub_A)
          if sub_D == []:
             leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)  
             print('3',leaf_value)
             node=Node([], feature_val=name, leaf_value=leaf_value)
             node.parent=parent
             parent.children.append(node)
             print('这是个叶子节点',f'他的父节点是{node.parent}他的节点D是{node.node_D}他的叶子节点值是:{node.leaf_value}')
             continue
          print('需要再分')
          child_node = self._build_tree(sub_D, sub_A, new_parent, name)
          # if child_node.
          # print('child_node',child_node.node_D)  
          # child_node.parent = self.now_parent  # 设置父节点  
          # self.now_parent.children.append(child_node)  # 使用标准的 children 而不是 childs  
          # print('children',self.now_parent.children)
      if node.parent!=None:
        print(f'一个节点彻底创建：\n他的父节点为{node.parent.node_D}他的父节点区分的标签索引是：{node.parent.feature_index}，他自己的节点D是：{node.node_D}他自己的区分索引是：{node.feature_index},他是否为叶子节点：{node.leaf_value}他的子节点为{node.children}')
        for child in node.children:
            print(child.node_D)
      else:
         print(f'一个节点彻底创建：他自己的节点D是：{node.node_D}他自己的区分索引是：{node.feature_index},他是否为叶子节点：{node.leaf_value}他的子节点为')
         for child in node.children:
            print(child.node_D)
            
      return child_node
    # 辅助函数：打印决策树（用于调试）  
def print_tree( node, level=0):  
    print(" " * level * 4 + f"Node_D_val: {node.feature_val if node.node_D else []}, Feature Index: {node.feature_index}, node: {node.children}, Leaf Value: {node.leaf_value}")  
    if node.children==[]:
      return 0
    for child in node.children:  
        print_tree(child, level + 1)  
    return 0
# D = [  
#     # 样本1: 色泽=青绿, 根蒂=蜷缩, 敲声=浊响, 纹理=清晰, 脐部=凹陷, 触感=硬滑, 是否好瓜=是  
#     ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑', '是'],  
#     # 样本2: 色泽=乌黑, 根蒂=稍蜷, 敲声=沉闷, 纹理=稍糊, 脐部=稍凹, 触感=软粘, 是否好瓜=否  
#     ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘', '否'],  
#     # 样本3: 色泽=乌黑, 根蒂=蜷缩, 敲声=沉闷, 纹理=清晰, 脐部=凹陷, 触感=硬滑, 是否好瓜=是  
#     ['乌黑', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '是'],  
#     # 样本4: 色泽=浅白, 根蒂=硬挺, 敲声=清脆, 纹理=模糊, 脐部=平坦, 触感=硬滑, 是否好瓜=否  
#     ['浅白', '硬挺', '清脆', '模糊', '平坦', '硬滑', '否'],  
#     # 样本5: 色泽=青绿, 根蒂=稍蜷, 敲声=浊响, 纹理=清晰, 脐部=稍凹, 触感=硬滑, 是否好瓜=是  
#     ['青绿', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', '是'],  
#     # 样本6: 色泽=乌黑, 根蒂=硬挺, 敲声=浊响, 纹理=清晰, 脐部=平坦, 触感=软粘, 是否好瓜=否  
#     ['乌黑', '硬挺', '浊响', '清晰', '平坦', '软粘', '否'],  
#     # 样本7: 色泽=浅白, 根蒂=蜷缩, 敲声=沉闷, 纹理=清晰, 脐部=凹陷, 触感=硬滑, 是否好瓜=是  
#     ['浅白', '蜷缩', '沉闷', '清晰', '凹陷', '硬滑', '是'],  
#     # 样本8: 色泽=青绿, 根蒂=硬挺, 敲声=清脆, 纹理=稍糊, 脐部=平坦, 触感=软粘, 是否好瓜=否  
#     ['青绿', '硬挺', '清脆', '稍糊', '平坦', '软粘', '否']  
# ]
# # # 正常构造
# A = ['色泽','根蒂','敲声','纹理','脐部','触感']
# # # 检查2类返回
# # # A = [0]  
# # # 3类返回，以稍倦之后的青绿节点为例，删去D中的样本5；但实际是由于在划分过程中，如稍倦+青绿的组合样本不存在，算法就不会创建相关节点，好在一个节点的叶子节点取值的判定规则并没有被打破，3类错误采用父节点的多数规则，所以不会出错
# # # 但需要注意的是在设置predict时
# # # D[4]=['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', '是']
# # print(D)
# tree_gen = TreeGenerate(D, A) 

# print_tree(tree_gen.root)

# # print_tree(tree_gen.root)
# Node_D_val: 根蒂, Feature Index: 1, node: [<__main__.Node object at 0x000001C7BAA68390>, <__main__.Node object at 0x000001C7B9DE4F98>, <__main__.Node object at 0x000001C7B9DE4C18>], Leaf Value: None
#     Node_D_val: 蜷缩, Feature Index: None, node: [], Leaf Value: 是
#     Node_D_val: 稍蜷, Feature Index: 0, node: [<__main__.Node object at 0x000001C7B9DE4EB8>, <__main__.Node object at 0x000001C7B9DE4358>], Leaf Value: None
#         Node_D_val: 乌黑, Feature Index: None, node: [], Leaf Value: 否
#         Node_D_val: 青绿, Feature Index: None, node: [], Leaf Value: 是
#     Node_D_val: 硬挺, Feature Index: None, node: [], Leaf Value: 否


In [4]:
D = [  
    # 样本1: 色泽=青绿, 根蒂=蜷缩, 敲声=浊响, 纹理=清晰, 脐部=凹陷, 触感=硬滑, 是否好瓜=是  
    ['1', '1', '1', '是'],  
    # 样本2: 色泽=乌黑, 根蒂=稍蜷, 敲声=沉闷, 纹理=稍糊, 脐部=稍凹, 触感=软粘, 是否好瓜=否  
    ['1', '2', '1','否'],  
    # 样本3: 色泽=乌黑, 根蒂=蜷缩, 敲声=沉闷, 纹理=清晰, 脐部=凹陷, 触感=硬滑, 是否好瓜=是  
    ['2', '2', '3' ,'是'],  
]
# # 正常构造
A = ['色泽','根蒂','敲声']
# # 检查2类返回
# # A = [0]  
# # 3类返回，以稍倦之后的青绿节点为例，删去D中的样本5；但实际是由于在划分过程中，如稍倦+青绿的组合样本不存在，算法就不会创建相关节点，好在一个节点的叶子节点取值的判定规则并没有被打破，3类错误采用父节点的多数规则，所以不会出错
# # 但需要注意的是在设置predict时
# # D[4]=['乌黑', '稍蜷', '浊响', '清晰', '稍凹', '硬滑', '是']
# print(D)
tree_gen = TreeGenerate(D, A) 

print_tree(tree_gen.root)
# print_tree(tree_gen.root)
# Node_D_val: 根蒂, Feature Index: 1, node: [<__main__.Node object at 0x000001C7BAA68390>, <__main__.Node object at 0x000001C7B9DE4F98>, <__main__.Node object at 0x000001C7B9DE4C18>], Leaf Value: None
#     Node_D_val: 蜷缩, Feature Index: None, node: [], Leaf Value: 是
#     Node_D_val: 稍蜷, Feature Index: 0, node: [<__main__.Node object at 0x000001C7B9DE4EB8>, <__main__.Node object at 0x000001C7B9DE4358>], Leaf Value: None
#         Node_D_val: 乌黑, Feature Index: None, node: [], Leaf Value: 否
#         Node_D_val: 青绿, Feature Index: None, node: [], Leaf Value: 是
#     Node_D_val: 硬挺, Feature Index: None, node: [], Leaf Value: 否

0 0.2516291673878229
0 ['1', '2']
创建根节点
name 1
sub_D [['1', '1', '1', '是'], ['1', '2', '1', '否']]
sub_A ['色泽', '根蒂', '敲声']
需要再分
1 1.0
1 ['1', '2']
创建中间节点
name 1
sub_D [['1', '1', '1', '是']]
sub_A ['色泽', '根蒂', '敲声']
需要再分
1 是
这是个叶子节点 他的父节点是<__main__.Node object at 0x000001D366F4D7F0>他的节点D是[['1', '1', '1', '是']]他的叶子节点值是:是
name 2
sub_D [['1', '2', '1', '否']]
sub_A ['色泽', '根蒂', '敲声']
需要再分
1 否
这是个叶子节点 他的父节点是<__main__.Node object at 0x000001D366F4D7F0>他的节点D是[['1', '2', '1', '否']]他的叶子节点值是:否
一个节点彻底创建：
他的父节点为[['1', '1', '1', '是'], ['1', '2', '1', '否'], ['2', '2', '3', '是']]他的父节点区分的标签索引是：0，他自己的节点D是：[['1', '1', '1', '是'], ['1', '2', '1', '否']]他自己的区分索引是：1,他是否为叶子节点：None他的子节点为[<__main__.Node object at 0x000001D366F4DA20>, <__main__.Node object at 0x000001D366F4DAC8>]
[['1', '1', '1', '是']]
[['1', '2', '1', '否']]
name 2
sub_D [['2', '2', '3', '是']]
sub_A ['色泽', '根蒂', '敲声']
需要再分
1 是
这是个叶子节点 他的父节点是<__main__.Node object at 0x000001D366F4D748>他的节点D是[['2', '2', '3', '是']]他的叶子节点值是:是
一个节点彻底创建：他自己的节点D是：[['1',

0

In [5]:
class classifyer:
    def __init__(self,tree,X,y):
        self.tree = tree
        self.X = X
        self.y = y
        self.predict_result=[]
        print('分类结果：',self.predict(self.X,tree.root))
    # classify=classifyer(tree=tree_gen,X=[['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑'],['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']],y=['是','否'])
    # ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑']
    # 1
    # p匹配上了 蜷缩
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 1
    # p匹配上了 稍蜷
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 0
    # p匹配上了 乌黑
    # ['是', '否']

    def predict(self,now_x,now_node):
        for x in now_x:
            
            # 如果是叶子节点，返回叶节点的取值。其实这个判断仅在根节点是叶子节点的特殊情况才会用到。如果考虑删除下面那句，又必然会多套一层函数，所以差别不是很大
            if now_node.leaf_value!=None:
                self.predict_result.append(now_node.leaf_value)
            print(x)
            feature_index=now_node.feature_index #获取本节点特征选取的index
            print(feature_index)
            for child in now_node.children:
                # 仅进入对应取值的分支
                if x[feature_index] == child.feature_val:
                    print('p匹配上了',child.feature_val)
                    # 如果孩子是叶节点，返回叶节点的取值
                    if child.leaf_value != None:
                        self.predict_result.append(child.leaf_value)
                    else:
                        
                        self.predict([x],child)
        
        return self.predict_result
        

In [7]:
classify=classifyer(tree=tree_gen,X=[['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑'],['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']],y=['是','否'])

['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑']
0
['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
0
分类结果： []


实验请将决策树代码移动至DecisionTree4.py文件，运行下面两个cell。支持调换剪枝策略。

In [6]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from package_py import DecisionTree4
import pandas as pd  
import numpy as np
from sklearn.preprocessing import StandardScaler  # 可选，用于标准化特征  
import matplotlib.pyplot as plt 

# 一次十折验证
def evaluate_classifier(classifier, X, y,k):
    """
    使用k折交叉验证评估分类器的性能
    
    参数:
    classifier: 分类器对象，需要实现fit和predict方法
    X: 特征矩阵，pandas DataFrame格式
    y: 标签向量，pandas Series格式
    k: 交叉验证的折数
    
    返回:
    mean_accuracy: 平均准确率
    std_accuracy: 准确率的标准差
    f1: F1分数
    """
    # 设置十折交叉验证  
    kf = KFold(n_splits=10, shuffle=True, random_state=42)  # shuffle=True表示在分割前打乱数据  
    scores = []
    # 遍历交叉验证的每一折  
    for fold, (train_index, test_index) in enumerate(kf.split(X)):  
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]  
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        # 首先将pandas读取的数据转化为array# 然后转化为list形式
        X_train = np.array(X_train).tolist()
        y_train = np.array(y_train).tolist()

        X_test = np.array(X_test).tolist()
        y_test = np.array(y_test).tolist()

        kd=classifier.KdTree(X_train, y_train)
        
        # # 训练分类器
        # classifier.fit(kd)

       
        y_pred = classifier.predicts(kd,X_test,k)
        accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
        f1 = f1_score(y_test, y_pred, average='macro')
        scores.append(accuracy)
        # print(X_test)
        # print(y_test)
        # print(y_pred)
        # print("/n")

    # print(scores)
    mean_accuracy = np.mean(scores)
    std_accuracy = np.std(scores)
    return mean_accuracy, std_accuracy,f1
from sklearn.model_selection import KFold  
from sklearn.metrics import accuracy_score, f1_score  
import numpy as np  
  
def evaluate_classifier_multiple_times(classifier, X, y, scissors=False,n_iterations=10):  
    all_accuracies = []  
    all_f1_scores = []  
  
    for iteration in range(n_iterations):  
        # 设置十折交叉验证，每次使用不同的random_state  
        kf = KFold(n_splits=10, shuffle=True, random_state=42 + iteration * 10)  # 使用迭代次数作为随机种子  
        scores = []  
        f1_scores_iter = []  
  
        # 遍历交叉验证的每一折  
        for fold, (train_index, test_index) in enumerate(kf.split(X)):  
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]  
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]  
  
            # 将pandas读取的数据转化为list形式
            X_train = X_train.values.tolist()  
            y_train = y_train.values.tolist()  
            X_test = X_test.values.tolist()  
            y_test = y_test.values.tolist()  
            index_list = list(range(len(X_train[0])))

            combined_list = [[*X_train[i], y_train[i]] for i in range(len(X_train))]
            
            tree = classifier.TreeGenerate(combined_list,index_list)  

            if scissors==True:

                tree.bhd_tree(tree.root)

            predict=classifier.predicts(tree, X_test, y_test) 
            y_pred =predict.predict(X_test,tree.root)
            # print('y_pred',y_pred,len(y_pred))
            # print('y_test',y_test,len(y_test))
            accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)  
            f1 = f1_score(y_test, y_pred, average='macro')  
            scores.append(accuracy)  
            f1_scores_iter.append(f1)  
  
        mean_accuracy = np.mean(scores)  
        std_accuracy = np.std(scores)  
        mean_f1 = np.mean(f1_scores_iter)  
        print(f'第{iteration}次',mean_accuracy)  
        all_accuracies.append(mean_accuracy)  
        all_f1_scores.append(mean_f1)  
  
        # print(f"Iteration {iteration + 1}: Mean Accuracy = {mean_accuracy:.4f}, Std Accuracy = {std_accuracy:.4f}, Mean F1 Score = {mean_f1:.4f}")  
  
    overall_mean_accuracy = np.mean(all_accuracies)  
    overall_std_accuracy = np.std(all_accuracies)  
    overall_mean_f1 = np.mean(all_f1_scores)  
  
    return overall_mean_accuracy, overall_std_accuracy, overall_mean_f1  
  
# # 示例调用  
# # classifier_instance = YourClassifier()  # 替换为你的分类器实例  
# # X = your_X_data  # 替换为你的特征数据  
# # y = your_y_data  # 替换为你的标签数据  
# # k = your_k_value  # 替换为你的k值  
# # evaluate_classifier_multiple_times(classifier_instance, X, y, k)


In [7]:
from package_py import DecisionTree4

    
# file_paths =[ "data\\gla.xls"]  # 实际文件路径
file_paths =[ "data\\bal.xls", "data\\gla.xls", "data\\hay.xls", "data\\iri.xls", "data\\new.xls", "data\\win.xls", "data\\zoo.xls"]  # 实际文件路径
# mean_accuracys=[]
for i in range(len(file_paths)):
    file_path=file_paths[i]

    data = pd.read_excel(file_path, header=None)  
    # 将数据分为特征和标签  
    X = data.iloc[:, :-1]  # 前4列是特征  
    y = data.iloc[:, -1]   # 最后一列是分类标签  
    
    # 数据标准hua
    scaler = StandardScaler()  
    X_scaled = scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

    mean_accuracy,std_accuracy,f1=evaluate_classifier_multiple_times(DecisionTree4,X_scaled_df,y)
    # mean_accuracys.append(mean_accuracy)

    # 使用 f-string 格式化输出  
    print(f'剪枝前{file_path} \n  mean_accuracy: {mean_accuracy:.3f} std_accuracy: {std_accuracy:.3f} f1: {f1:.3f}')

    mean_accuracy,std_accuracy,f1=evaluate_classifier_multiple_times(DecisionTree4,X_scaled_df,y,True)
    
    print(f'剪枝后{file_path} \n  mean_accuracy: {mean_accuracy:.3f} std_accuracy: {std_accuracy:.3f} f1: {f1:.3f}')


第0次 0.6384536610343061
第1次 0.6496671786994368
第2次 0.6560163850486431
第3次 0.6459293394777265
第4次 0.6462621607782898
第5次 0.6576804915514594
第6次 0.6495647721454174
第7次 0.632258064516129
第8次 0.6543778801843317
第9次 0.6545314900153609
剪枝前data\bal.xls 
  mean_accuracy: 0.648 std_accuracy: 0.008 f1: 0.448
第0次 0.5549923195084486
第1次 0.5632616487455198
第2次 0.6016129032258064
第3次 0.5513312852022529
第4次 0.5791090629800307
第5次 0.5443676395289299
第6次 0.5744751664106504
第7次 0.5138760880696365
第8次 0.55378904249872
第9次 0.5664874551971326
剪枝后data\bal.xls 
  mean_accuracy: 0.560 std_accuracy: 0.022 f1: 0.368
第0次 0.4004329004329004
第1次 0.36904761904761907
第2次 0.40649350649350646
第3次 0.42857142857142855
第4次 0.41991341991341996
第5次 0.4227272727272727
第6次 0.3683982683982684
第7次 0.40562770562770567
第8次 0.42510822510822505
第9次 0.40562770562770567
剪枝前data\gla.xls 
  mean_accuracy: 0.405 std_accuracy: 0.020 f1: 0.204
第0次 0.4004329004329004
第1次 0.36904761904761907
第2次 0.40649350649350646
第3次 0.42857142857142855
第

调库

In [1]:
import pandas as pd  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import KFold, cross_val_score, cross_validate  
from sklearn.metrics import accuracy_score, f1_score  
  
# 文件路径列表  
file_paths = [  
    "data\\bal.xls", "data\\gla.xls", "data\\hay.xls",   
    "data\\iri.xls", "data\\new.xls", "data\\win.xls", "data\\zoo.xls"  
]  
  
# 初始化结果字典  
results = {}  
  
# 对每个数据集进行交叉验证  
for file_path in file_paths:  
    # 读取Excel文件  
    df = pd.read_excel(file_path)  
      
    # 分离特征和标签  
    X = df.iloc[:, :-1]  # 所有行，除了最后一列的所有列（特征）  
    y = df.iloc[:, -1]   # 所有行，最后一列（标签）  
      
    # 初始化决策树分类器  
    clf = DecisionTreeClassifier(random_state=42)  
      
    # 设置KFold交叉验证  
    kf = KFold(n_splits=10, shuffle=True, random_state=42)  
      
    # 计算准确度  
    accuracies = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')  
    mean_accuracy = accuracies.mean()  
      
    # 计算F1分数（假设是多分类问题，使用macro平均）  
    f1_scores = cross_val_score(clf, X, y, cv=kf, scoring='f1_macro')  
    mean_f1 = f1_scores.mean()  
      
    # 将结果保存到字典中  
    # 这里使用文件名（不带路径和扩展名）作为键  
    file_name = file_path.split("\\")[-1].split(".")[0]  
    results[file_name] = {  
        'mean_accuracy': mean_accuracy,  
        'mean_f1': mean_f1  
    }  
  
# 将结果转换为DataFrame以便于查看（可选）  
import pandas as pd  
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['mean_accuracy', 'mean_f1'])  
print(results_df)

d:\anaconda\envs\TF\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
d:\anaconda\envs\TF\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
d:\anaconda\envs\TF\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
d:\anaconda\envs\TF\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
d:\anaconda\envs\TF\lib\site-packages\sklearn\ut

     mean_accuracy   mean_f1
bal       0.777343  0.573655
gla       0.690909  0.611936
hay       0.817917  0.843665
iri       0.926190  0.908986
new       0.948918  0.934580
win       0.910131  0.908474
zoo       0.950000  0.920437


d:\anaconda\envs\TF\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  FutureWarning,
d:\anaconda\envs\TF\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  FutureWarning,
d:\anaconda\envs\TF\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  FutureWarning,
d:\anaconda\envs\TF\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float']. An error will be raised in 1.2.
  FutureWarning,
d:\anaconda\envs\TF\lib\site-packages\sklearn\utils\validation.py:16

存档（最基础的决策树）

In [1]:
import math
from collections import Counter  
class Node(object):
  # def __init__(self,node_D,feature_index=None,parent=None,children=[],leaf_value=None):
  
    def __init__(self, node_D, feature_val=None, feature_index=None, parent=None, children=None, leaf_value=None):  
        self.node_D = node_D #节点数据  
        self.feature_val=feature_val #属性值
        self.feature_index = feature_index #属性索引 
        self.parent = parent  #父节点
        self.children = children if children is not None else [] # 子节点们  
        self.leaf_value = leaf_value #叶子节点的值（好瓜坏瓜）


class TreeGenerate(object):
  def __init__(self,D,A):

     self.D=D #数据集
     self.A=A #属性集
     self.root=None
     self._build_tree(D,A)

    # calculate_pk([[1,1],[2,2]])
    # {'1': 0.5, '2': 0.5}
  def calculate_pk(self,D):
    A=[]
    for i in range(len(D)): 
      # 提取最后一列数据
      A.append(D[i][-1])
        
    # 创建一个字典来存储每个值（转换为字符串后）出现的次数  
    count_dict = {}     
    # 统计每个值（转换为字符串后）出现的次数  
    for item in A:  
        # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
        # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
        item_str = str(item)  
        if item_str in count_dict:  
            count_dict[item_str] += 1  
        else:  
            count_dict[item_str] = 1       
    # 计算列表中元素的总数  
    total_count = len(A)         
    # 计算每个值（转换为字符串后）出现的次数占总次数的比例  
    proportion_dict = {item_str: count / total_count for item_str, count in count_dict.items()}     
    # 打印结果  
    return proportion_dict

  # print(calculate_Entropy(['是','是','是','是','是','是','是','是','否','否','否','否','否','否','否','否','否']))
  # 0.9975025463691153
  def calculate_Entropy(self,D):

    p = self.calculate_pk(D)
    entropy = 0
    for i in p.values():
      entropy += i*math.log(i,2)
    return -entropy
  # 计算第二个（索引1）属性的信息增益
  # calculate_Gain(0,[[1,2,3,0],[2,3,1,0],[3,1,2,0],[1,2,3,0],[2,3,1,0],[3,1,2,1]],1)
  # 0.3166890883150208 ['2', '3', '1']
  # [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

  def calculate_Gain(self,D,a):
      if D==[]:
        print('D is empty')
        return 0
      if a<0 or a>=len(D[0]):
        print('a is not in D')
        return 0
      
      Ent_D = self.calculate_Entropy(D)
      # print(Ent_D)
      Ent_Di={}
      count_dict = {}  # 记录指定属性的取值的次数
      attribute_name=[]   # 记录指定属性的取值
      
      # 统计每个值（转换为字符串后）出现的次数  
      for item in D:  
      # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
      # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
        item_str = str(item[a])  
        if item_str in count_dict:  
          count_dict[item_str] += 1  
          Ent_Di[item_str].append(item)
        else:  
          count_dict[item_str] = 1
          Ent_Di[item_str]=[]
          Ent_Di[item_str].append(item)
          attribute_name.append(item_str)       

      # 计算列表中元素的总数
      total_count = len(D)
      # 计算每个值（转换为字符串后）出现的次数占总次数的比例
      attribute_weight = [count / total_count for count in count_dict.values()]
      # print(attribute_weight)
      # print(attribute_name)
      gain=Ent_D
      for i in range(len(attribute_name)):
        gain-=attribute_weight[i]*self.calculate_Entropy(Ent_Di[attribute_name[i]])

      return gain,attribute_name

  # 选择最大信息增益属性
  # choose_A(0,[[2,2,3,0],[2,2,1,0],[2,2,2,0],[1,2,3,0],[3,3,3,0],[3,1,2,1]],[0,1,2])
  # (1, ['2', '3', '1'])
  def choose_A(self,D,A):
        max_gain=0
        for i in range(len(A)):
          gain,attribute_name=self.calculate_Gain(D,i)
          # 考虑等于0，是因为在外层遍历中并未考虑D在指定特征列的取值已经全部相同的情况
          if gain>=max_gain:
            max_gain=gain
            max_attribute_index=i
            max_attribute_name=attribute_name
          

        return max_attribute_index,max_attribute_name,max_gain
  # def __init__(self,node_D,feature_index,parent=None,childs=[],leaf_value=None):
  # self.node_D=node_D #节点数据
  # self.feature_index = feature_index #属性索引
  # self.parent = parent #父节点
  # self.childs = childs #子节点们
  # self.leaf_value = leaf_value  #叶子节点的值（好瓜坏瓜）
  # 递归构建决策树  
  def _build_tree(self, D, A, parent=None, now_feature_val=None):  
      # 如果所有样本都属于同一类，则返回叶子节点  
      if len(set([item[-1] for item in D])) == 1:  
          leaf_value = D[0][-1] 
          node=Node(D, feature_val=now_feature_val,leaf_value=leaf_value)  
          node.parent=parent
          parent.children.append(node)
          return node
      # 如果没有更多特征可以选择，则返回叶子节点（多数类投票）  
      if not A:  
          leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)  
          node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
          node.parent=parent
          parent.children.append(node)
          return node  
      if D==[]:
        leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)
        node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
        node.parent=parent
        parent.children.append(node)
        return node
      # 选择最佳特征  
      feature_index, feature_name ,feature_gain= self.choose_A(D, A)  
      if feature_gain==0:
        leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)
        node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
        node.parent=parent
        parent.children.append(node)
        return node
      # 创建根节点 
      if self.root==None: 
        root = Node(D, feature_val=self.A[feature_index],feature_index=feature_index)
        self.root=root
        new_parent=root
        node=root
      else:
        node=Node(D, feature_val=now_feature_val,feature_index=feature_index)
        parent.children.append(node)
        node.parent=parent
        new_parent=node
      
      # 递归地为每个特征值创建子节点  
      for name in feature_name:  
          # 使用 math.isclose 进行比较，允许一个小的容差  
          sub_D = [item for item in D if math.isclose(item[feature_index], float(name))]
          # sub_D = [item for item in D if item[feature_index] == name]  
          sub_A = [i for i in A if i != feature_index]  # 移除已使用的特征  
          if sub_D == []:
             leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)  
             node=Node([], feature_val=name, leaf_value=leaf_value)
            #  1111111111
             if parent!=None:
                node.parent=parent
                parent.children.append(node)
             continue
          child_node = self._build_tree(sub_D, sub_A, new_parent, name)
          # if child_node.
          # print('child_node',child_node.node_D)  
          # child_node.parent = self.now_parent  # 设置父节点  
          # self.now_parent.children.append(child_node)  # 使用标准的 children 而不是 childs  
          # print('children',self.now_parent.children)

            
      return child_node
  # 剪枝
  def check_accuracy(self,D,value):
    # 计算预测值与真实值的差异
    accuracy = sum(1 for d in D if d[-1] == value) / len(D)
    return accuracy
  
  # 如果想要改动剪枝阈值在此设置
  def is_del(self,node):
    now_accuracy=self.check_accuracy(node.node_D,node.leaf_value)
    # 查看父节点多数原则
    last_column_values = [sublist[-1] for sublist in node.parent.node_D]   
    counter = Counter(last_column_values)   
    parent_leaf_value,_ = counter.most_common(1)[0] 
    # 剪枝后准确率
    after_accuracy=self.check_accuracy(node.node_D,parent_leaf_value)
    
    # 此处的剪枝策略是：如果剪枝后准确率比剪枝前低，则剪枝
    if now_accuracy >after_accuracy:
        return False
    else:
        return True
  def del_node(self,node):
    # 删除子节点，并未释放空间，remove只会删除第一个元素，但这并不会造成问题，因为子节点必然互不相同
    return node.parent.children.remove(node)
  def bhd_tree(self,now_node):

    # 如果已经是根节点并且没有子节点，退出
    if now_node == self.root and now_node.children == []:

      return True
    # 如果是叶子节点，则直接考虑是否剪枝
    if now_node.leaf_value != None:

      if self.is_del(now_node):

        self.del_node(now_node)
     
      return True
    # 下面是针对中间节点的递归
    
    for child in reversed(now_node.children):
        self.bhd_tree(child)
    # 如果中间节点的所有子节点都没了，那给该中间节点赋予leaf_value,并向上追溯
    if now_node.children==[]:
        last_column_values = [sublist[-1] for sublist in now_node.node_D]   
        counter = Counter(last_column_values)   
        leaf_value,_ = counter.most_common(1)[0] 
        now_node.leaf_value=leaf_value
        self.bhd_tree(now_node)

    # 辅助函数：打印决策树（用于调试）  
def print_tree( node, level=0):  
    print(" " * level * 4 + f"Node_D_val: {node.feature_val if node.node_D else []}, Feature Index: {node.feature_index}, node_node_D: {node.node_D}, node_children: {node.children}, Leaf Value: {node.leaf_value}")  
    if node.children==[]:
      return 0
    for child in node.children:  
        print_tree(child, level + 1)  
    return 0

class predicts:
    def __init__(self,tree,X,y):
        self.tree = tree
        self.X = X
        self.y = y
        self.predict_result=[]
        # self.predict(self.X,tree.root)
    # classify=classifyer(tree=tree_gen,X=[['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑'],['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']],y=['是','否'])
    # ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑']
    # 1
    # p匹配上了 蜷缩
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 1
    # p匹配上了 稍蜷
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 0
    # p匹配上了 乌黑
    # ['是', '否']
     
    def predict(self,now_x,now_node):
      
        for x in now_x:
            # 如果是叶子节点，返回叶节点的取值。其实这个判断仅在根节点是叶子节点的特殊情况才会用到。如果考虑删除下面那句，又必然会多套一层函数，所以差别不是很大
            if now_node.leaf_value!=None:
                self.predict_result.append(int(now_node.leaf_value))
  
            feature_index=now_node.feature_index #获取本节点特征选取的index
    
            for child in now_node.children:
                # 仅进入对应取值的分支
                if math.isclose(x[feature_index],float(child.feature_val)):
               
                    # 如果孩子是叶节点，返回叶节点的取值
                    if child.leaf_value != None:
                        self.predict_result.append(int(child.leaf_value))
                    else:
                        
                        self.predict([x],child)
                    break
                  # 1111111111
                else:
                  if child==now_node.children[-1]:
                    
                    # 提取每个子列表的最后一个元素  
                    last_column_values = [sublist[-1] for sublist in now_node.node_D]  
                      
                    # 使用 Counter 统计频次  
                    counter = Counter(last_column_values)  
                      
                    # 找到频次最高的元素及其频次  
                    most_common_value, most_common_count = counter.most_common(1)[0] 

                    self.predict_result.append(int(most_common_value))
        
        return self.predict_result



        


C4.5版本决策树

In [12]:
import math
from collections import Counter  
class Node(object):
  # def __init__(self,node_D,feature_index=None,parent=None,children=[],leaf_value=None):
  
    def __init__(self, node_D, feature_val=None, feature_index=None, parent=None, children=None, leaf_value=None):  
        self.node_D = node_D #节点数据  
        self.feature_val=feature_val #属性值
        self.feature_index = feature_index #属性索引 
        self.parent = parent  #父节点
        self.children = children if children is not None else [] # 子节点们  
        self.leaf_value = leaf_value #叶子节点的值（好瓜坏瓜）


class TreeGenerate(object):
  def __init__(self,D,A):

     self.D=D #数据集
     self.A=A #属性集
     self.root=None
     self._build_tree(D,A)

    # calculate_pk([[1,1],[2,2]])
    # {'1': 0.5, '2': 0.5}
  def calculate_pk(self,D):
    A=[]
    for i in range(len(D)): 
      # 提取最后一列数据
      A.append(D[i][-1])
        
    # 创建一个字典来存储每个值（转换为字符串后）出现的次数  
    count_dict = {}     
    # 统计每个值（转换为字符串后）出现的次数  
    for item in A:  
        # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
        # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
        item_str = str(item)  
        if item_str in count_dict:  
            count_dict[item_str] += 1  
        else:  
            count_dict[item_str] = 1       
    # 计算列表中元素的总数  
    total_count = len(A)         
    # 计算每个值（转换为字符串后）出现的次数占总次数的比例  
    proportion_dict = {item_str: count / total_count for item_str, count in count_dict.items()}     
    # 打印结果  
    return proportion_dict

  # print(calculate_Entropy(['是','是','是','是','是','是','是','是','否','否','否','否','否','否','否','否','否']))
  # 0.9975025463691153
  def calculate_Entropy(self,D):

    p = self.calculate_pk(D)
    entropy = 0
    for i in p.values():
      entropy += i*math.log(i,2)
    return -entropy
  # 计算第二个（索引1）属性的信息增益
  # calculate_Gain(0,[[1,2,3,0],[2,3,1,0],[3,1,2,0],[1,2,3,0],[2,3,1,0],[3,1,2,1]],1)
  # 0.3166890883150208 ['2', '3', '1']
  # [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

  def calculate_Gain(self,D,a):
      if D==[]:
        print('D is empty')
        return 0
      if a<0 or a>=len(D[0]):
        print('a is not in D')
        return 0
      Ent_D = self.calculate_Entropy(D)
      # print(Ent_D)
      Ent_Di={}
      count_dict = {}  # 记录指定属性的取值的次数
      attribute_name=[]   # 记录指定属性的取值
      # 统计每个值（转换为字符串后）出现的次数  
      for item in D:  
      # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
      # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
        item_str = str(item[a])  
        if item_str in count_dict:  
          count_dict[item_str] += 1  
          Ent_Di[item_str].append(item)
        else:  
          count_dict[item_str] = 1
          Ent_Di[item_str]=[]
          Ent_Di[item_str].append(item)
          attribute_name.append(item_str)       
      # 计算列表中元素的总数
      total_count = len(D)
      # 计算每个值（转换为字符串后）出现的次数占总次数的比例
      attribute_weight = [count / total_count for count in count_dict.values()]
      # print(attribute_weight)
      # print(attribute_name)
      gain=Ent_D
      # C4.5改动之处
      iv=0
      for i in range(len(attribute_name)):
        gain -=attribute_weight[i]*self.calculate_Entropy(Ent_Di[attribute_name[i]])
        # 计算增益率
        iv -=attribute_weight[i]*math.log(attribute_weight[i],2)
      if iv!=0:
        gain_ratio=gain/iv
      # 此时便是A属性取值一样的情况
      else:
        gain_ratio=0
      return gain,attribute_name,gain_ratio
 

  # 选择最大信息增益属性
  # choose_A(0,[[2,2,3,0],[2,2,1,0],[2,2,2,0],[1,2,3,0],[3,3,3,0],[3,1,2,1]],[0,1,2])
  # (1, ['2', '3', '1'])
  def choose_A_by_c45(self,D,A):
        
        gains=[]
        attribute_names=[]
        gain_ratios=[]
        for i in range(len(A)):
          gain,attribute_name,gain_ratio=self.calculate_Gain(D,i)
          
          gains.append(gain)
          attribute_names.append(attribute_name)
          gain_ratios.append(gain_ratio)
        # C4.5改动之处
        # 计算列表的平均值  
        average_gain = sum(gains) / len(gains)  
        #记录gain高于平均值的索引i_index
        i_index=[]
        for i in range(len(A)):

           if gains[i]>=average_gain:
              i_index.append(i)
        # 找到ratio最大的索引
        max_ratio=0
        max_index=0
        for i in i_index:
           if gain_ratios[i]>=max_ratio:
            max_index=i
        
        max_attribute_index=max_index
        max_attribute_name=attribute_names[max_index]
        max_ratio=gain_ratios[max_index]
        return max_attribute_index,max_attribute_name,max_ratio
  # def __init__(self,node_D,feature_index,parent=None,childs=[],leaf_value=None):
  # self.node_D=node_D #节点数据
  # self.feature_index = feature_index #属性索引
  # self.parent = parent #父节点
  # self.childs = childs #子节点们
  # self.leaf_value = leaf_value  #叶子节点的值（好瓜坏瓜）
  # 递归构建决策树  
  def _build_tree(self, D, A, parent=None, now_feature_val=None):  
      # 如果所有样本都属于同一类，则返回叶子节点  
      if len(set([item[-1] for item in D])) == 1:  
          leaf_value = D[0][-1]  
          
          node=Node(D, feature_val=now_feature_val,leaf_value=leaf_value)  
          node.parent=parent
          parent.children.append(node)
         

          return node
      # 如果没有更多特征可以选择，则返回叶子节点（多数类投票）  
      if not A:  
          leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)  

          node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
          node.parent=parent
          parent.children.append(node)
       
          return node  
      if D==[]:
 
        leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)
        node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
        node.parent=parent
        parent.children.append(node)
 
        return node
       

      # 选择最佳特征  # C4.5改动之处，名字
      feature_index, feature_name ,feature_gain= self.choose_A_by_c45(D, A)  
      # 如果D数据集中在属性a上取值一样，返回叶子节点
      if feature_gain==0:
        leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)
        node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
        node.parent=parent
        parent.children.append(node)

        return node

      # 创建根节点 
      if self.root==None: 

        root = Node(D, feature_val=self.A[feature_index],feature_index=feature_index)
        self.root=root
        new_parent=root
        node=root
      else:

        node=Node(D, feature_val=now_feature_val,feature_index=feature_index)
        parent.children.append(node)
        node.parent=parent
        new_parent=node
      
      # 递归地为每个特征值创建子节点  
      for name in feature_name:  

          
          # 使用 math.isclose 进行比较，允许一个小的容差  
          sub_D = [item for item in D if math.isclose(item[feature_index], float(name))]
          # sub_D = [item for item in D if item[feature_index] == name]  
          sub_A = [i for i in A if i != feature_index]  # 移除已使用的特征  

          if sub_D == []:
             leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)  

             node=Node([], feature_val=name, leaf_value=leaf_value)
            #  1111111111
             if parent!=None:
                node.parent=parent
                parent.children.append(node)

             continue

          child_node = self._build_tree(sub_D, sub_A, new_parent, name)
          # if child_node.
          # print('child_node',child_node.node_D)  
          # child_node.parent = self.now_parent  # 设置父节点  
          # self.now_parent.children.append(child_node)  # 使用标准的 children 而不是 childs  
          # print('children',self.now_parent.children)

            
      return child_node
  # 剪枝
  def check_accuracy(self,D,value):
    # 计算预测值与真实值的差异
    accuracy = sum(1 for d in D if d[-1] == value) / len(D)
    return accuracy
  

  # 如果想要改动剪枝阈值在此设置
  def is_del(self,node):
    now_accuracy=self.check_accuracy(node.node_D,node.leaf_value)
    # 查看父节点多数原则
    last_column_values = [sublist[-1] for sublist in node.parent.node_D]   
    counter = Counter(last_column_values)   
    parent_leaf_value,_ = counter.most_common(1)[0] 
    # 剪枝后准确率
    after_accuracy=self.check_accuracy(node.node_D,parent_leaf_value)
    
    # 此处的剪枝策略是：如果剪枝后准确率比剪枝前低，则剪枝
    if now_accuracy >after_accuracy:
        return False
    else:
        return True
  def del_node(self,node):
    # 删除子节点，并未释放空间，remove只会删除第一个元素，但这并不会造成问题，因为子节点必然互不相同
    return node.parent.children.remove(node)
  def bhd_tree(self,now_node):

    # 如果已经是根节点并且没有子节点，退出
    if now_node == self.root and now_node.children == []:

      return True
    # 如果是叶子节点，则直接考虑是否剪枝
    if now_node.leaf_value != None:

      if self.is_del(now_node):

        self.del_node(now_node)
     
      return True
    # 下面是针对中间节点的递归
    
    for child in reversed(now_node.children):
        self.bhd_tree(child)
    # 如果中间节点的所有子节点都没了，那给该中间节点赋予leaf_value,并向上追溯
    if now_node.children==[]:
        last_column_values = [sublist[-1] for sublist in now_node.node_D]   
        counter = Counter(last_column_values)   
        leaf_value,_ = counter.most_common(1)[0] 
        now_node.leaf_value=leaf_value
        self.bhd_tree(now_node)
    # 辅助函数：打印决策树（用于调试）  
def print_tree( node, level=0):  
    print(" " * level * 4 + f"Node_D_val: {node.feature_val if node.node_D else []}, Feature Index: {node.feature_index}, node: {node.children}, Leaf Value: {node.leaf_value}")  
    if node.children==[]:
      return 0
    for child in node.children:  
        print_tree(child, level + 1)  
    return 0

class predicts:
    def __init__(self,tree,X,y):
        self.tree = tree
        self.X = X
        self.y = y
        self.predict_result=[]
        # self.predict(self.X,tree.root)
    # classify=classifyer(tree=tree_gen,X=[['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑'],['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']],y=['是','否'])
    # ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑']
    # 1
    # p匹配上了 蜷缩
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 1
    # p匹配上了 稍蜷
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 0
    # p匹配上了 乌黑
    # ['是', '否']
     
    def predict(self,now_x,now_node):
       
        for x in now_x:
            # 如果是叶子节点，返回叶节点的取值。其实这个判断仅在根节点是叶子节点的特殊情况才会用到。如果考虑删除下面那句，又必然会多套一层函数，所以差别不是很大
            if now_node.leaf_value!=None:
                self.predict_result.append(int(now_node.leaf_value))
           
            feature_index=now_node.feature_index #获取本节点特征选取的index
            
            for child in now_node.children:
                # 仅进入对应取值的分支
                if math.isclose(x[feature_index],float(child.feature_val)):
                    
                    # 如果孩子是叶节点，返回叶节点的取值
                    if child.leaf_value != None:
                        self.predict_result.append(int(child.leaf_value))
                    else:
                        
                        self.predict([x],child)
                    break
                  # 1111111111
                else:
                  if child==now_node.children[-1]:
                    
                    # 提取每个子列表的最后一个元素  
                    last_column_values = [sublist[-1] for sublist in now_node.node_D]  
                      
                    # 使用 Counter 统计频次  
                    counter = Counter(last_column_values)  
                      
                    # 找到频次最高的元素及其频次  
                    most_common_value, most_common_count = counter.most_common(1)[0] 
                    
                    self.predict_result.append(int(most_common_value))
        
        return self.predict_result



        


CART版本决策树+剪枝

In [11]:
import math
from collections import Counter  
class Node(object):
  # def __init__(self,node_D,feature_index=None,parent=None,children=[],leaf_value=None):
  
    def __init__(self, node_D, feature_val=None, feature_index=None, parent=None, children=None, leaf_value=None):  
        self.node_D = node_D #节点数据  
        self.feature_val=feature_val #属性值
        self.feature_index = feature_index #属性索引 
        self.parent = parent  #父节点
        self.children = children if children is not None else [] # 子节点们  
        self.leaf_value = leaf_value #叶子节点的值（好瓜坏瓜）

def print_tree( node, level=0):  
    print(" " * level * 4 + f"Node_D_val: {node.feature_val if node.node_D else []}, Feature Index: {node.feature_index}, node: {node.children}, Leaf Value: {node.leaf_value}")  
    if node.children==[]:
      return 0
    for child in node.children:  
        print_tree(child, level + 1)  
    return 0
class TreeGenerate(object):
  def __init__(self,D,A):

     self.D=D #数据集
     self.A=A #属性集
     self.root=None
     self._build_tree(D,A)

    # calculate_pk([[1,1],[2,2]])
    # {'1': 0.5, '2': 0.5}
  def calculate_pk(self,D):
    A=[]
    for i in range(len(D)): 
      # 提取最后一列数据
      A.append(D[i][-1])
        
    # 创建一个字典来存储每个值（转换为字符串后）出现的次数  
    count_dict = {}     
    # 统计每个值（转换为字符串后）出现的次数  
    for item in A:  
        # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
        # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
        item_str = str(item)  
        if item_str in count_dict:  
            count_dict[item_str] += 1  
        else:  
            count_dict[item_str] = 1       
    # 计算列表中元素的总数  
    total_count = len(A)         
    # 计算每个值（转换为字符串后）出现的次数占总次数的比例  
    proportion_dict = {item_str: count / total_count for item_str, count in count_dict.items()}     
    # 打印结果  
    return proportion_dict

  # print(calculate_Entropy(['是','是','是','是','是','是','是','是','否','否','否','否','否','否','否','否','否']))
  # 0.9975025463691153
  def calculate_Entropy(self,D):

    p = self.calculate_pk(D)
    entropy = 0
    for i in p.values():
      entropy += i*math.log(i,2)
    return -entropy
  # 计算第二个（索引1）属性的信息增益
  # calculate_Gain(0,[[1,2,3,0],[2,3,1,0],[3,1,2,0],[1,2,3,0],[2,3,1,0],[3,1,2,1]],1)
  # 0.3166890883150208 ['2', '3', '1']
  # [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
  def calculate_Gini(self,D):
    p = self.calculate_pk(D)
    gini = 1
    for i in p.values():

      gini -= i**2
    return gini
  def calculate_Gini_Index(self,D,a):
      if D==[]:
        print('D is empty')
        return 0
      if a<0 or a>=len(D[0]):
        print('a is not in D')
        return 0
      
      Ent_Di={}
      count_dict = {}  # 记录指定属性的取值的次数
      attribute_name=[]   # 记录指定属性的取值
      
      # 统计每个值（转换为字符串后）出现的次数  
      for item in D:  
      # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
      # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
        item_str = str(item[a])  
        if item_str in count_dict:  
          count_dict[item_str] += 1  
          Ent_Di[item_str].append(item)
        else:  
          count_dict[item_str] = 1
          Ent_Di[item_str]=[]
          Ent_Di[item_str].append(item)
          attribute_name.append(item_str)       

      # 计算列表中元素的总数
      total_count = len(D)
      # 计算每个值（转换为字符串后）出现的次数占总次数的比例
      attribute_weight = [count / total_count for count in count_dict.values()]
      # print(attribute_weight)
      # print(attribute_name)
      gini_index=0
      for i in range(len(attribute_name)):
        gini_index+=attribute_weight[i]*self.calculate_Gini(Ent_Di[attribute_name[i]])

      return gini_index,attribute_name
  def calculate_Gain(self,D,a):
      if D==[]:
        print('D is empty')
        return 0
      if a<0 or a>=len(D[0]):
        print('a is not in D')
        return 0
      
      Ent_D = self.calculate_Entropy(D)
      # print(Ent_D)
      Ent_Di={}
      count_dict = {}  # 记录指定属性的取值的次数
      attribute_name=[]   # 记录指定属性的取值
      
      # 统计每个值（转换为字符串后）出现的次数  
      for item in D:  
      # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
      # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
        item_str = str(item[a])  
        if item_str in count_dict:  
          count_dict[item_str] += 1  
          Ent_Di[item_str].append(item)
        else:  
          count_dict[item_str] = 1
          Ent_Di[item_str]=[]
          Ent_Di[item_str].append(item)
          attribute_name.append(item_str)       

      # 计算列表中元素的总数
      total_count = len(D)
      # 计算每个值（转换为字符串后）出现的次数占总次数的比例
      attribute_weight = [count / total_count for count in count_dict.values()]
      # print(attribute_weight)
      # print(attribute_name)
      gain=Ent_D
      for i in range(len(attribute_name)):
        gain-=attribute_weight[i]*self.calculate_Entropy(Ent_Di[attribute_name[i]])

      return gain,attribute_name

  # 选择最大信息增益属性
  # choose_A(0,[[2,2,3,0],[2,2,1,0],[2,2,2,0],[1,2,3,0],[3,3,3,0],[3,1,2,1]],[0,1,2])
  # (1, ['2', '3', '1'])
  def choose_A_by_Gini(self,D,A):
        # 由于最大的gini_index不会超过A属性取值的个数
        min_gini=len(A)
        for i in range(len(A)):
          gini_index,attribute_name=self.calculate_Gini_Index(D,i)
          gain,_=self.calculate_Gain(D,i)
          # 考虑等于0，是因为在外层遍历中并未考虑D在指定特征列的取值已经全部相同的情况
          if gini_index<=min_gini:
            min_gini=gini_index
            min_attribute_index=i
            min_attribute_name=attribute_name
        
 
        return min_attribute_index,min_attribute_name,min_gini,gain
  # def __init__(self,node_D,feature_index,parent=None,childs=[],leaf_value=None):
  # self.node_D=node_D #节点数据
  # self.feature_index = feature_index #属性索引
  # self.parent = parent #父节点
  # self.childs = childs #子节点们
  # self.leaf_value = leaf_value  #叶子节点的值（好瓜坏瓜）
  # 递归构建决策树  
  def _build_tree(self, D, A, parent=None, now_feature_val=None):  
      # 如果所有样本都属于同一类，则返回叶子节点  
      if len(set([item[-1] for item in D])) == 1:  
          leaf_value = D[0][-1]  

          node=Node(D, feature_val=now_feature_val,leaf_value=leaf_value)  
          node.parent=parent
          parent.children.append(node)
          

          return node
      # 如果没有更多特征可以选择，则返回叶子节点（多数类投票）  
      if not A:  
          leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)  
     
          node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
          node.parent=parent
          parent.children.append(node)
          
          return node  
      if D==[]:

        leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)
        node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
        node.parent=parent
        parent.children.append(node)
        
        return node
       

      # 选择最佳特征  
      feature_index, feature_name ,min_gini,feature_gain= self.choose_A_by_Gini(D, A) 

      if feature_gain==0:
        leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)
        node=Node(D, feature_val=now_feature_val, leaf_value=leaf_value)  
        node.parent=parent
        parent.children.append(node)
        
        return node
      
      # 创建根节点 
      if self.root==None: 
       
        root = Node(D, feature_val=self.A[feature_index],feature_index=feature_index)
        self.root=root
        new_parent=root
        node=root
      else:
        
        node=Node(D, feature_val=now_feature_val,feature_index=feature_index)
        parent.children.append(node)
        node.parent=parent
        new_parent=node
      
      # 递归地为每个特征值创建子节点  
      for name in feature_name:  
    
          
          # 使用 math.isclose 进行比较，允许一个小的容差  
          sub_D = [item for item in D if math.isclose(item[feature_index], float(name))]
          # sub_D = [item for item in D if item[feature_index] == name]  
          sub_A = [i for i in A if i != feature_index]  # 移除已使用的特征  
       
          if sub_D == []:
             leaf_value = max(self.calculate_pk(D), key=self.calculate_pk(D).get)  
          
             node=Node([], feature_val=name, leaf_value=leaf_value)
            #  1111111111
             if parent!=None:
                node.parent=parent
                parent.children.append(node)
             
             continue
          
          child_node = self._build_tree(sub_D, sub_A, new_parent, name)
          # if child_node.
          # print('child_node',child_node.node_D)  
          # child_node.parent = self.now_parent  # 设置父节点  
          # self.now_parent.children.append(child_node)  # 使用标准的 children 而不是 childs  
          # print('children',self.now_parent.children)
      
            
      return child_node
  # 剪枝
  def check_accuracy(self,D,value):
    # 计算预测值与真实值的差异
    accuracy = sum(1 for d in D if d[-1] == value) / len(D)
    return accuracy
  def is_del(self,node):
    now_accuracy=self.check_accuracy(node.node_D,node.leaf_value)
    # 查看父节点多数原则
    last_column_values = [sublist[-1] for sublist in node.parent.node_D]   
    counter = Counter(last_column_values)   
    parent_leaf_value,_ = counter.most_common(1)[0] 
    # 剪枝后准确率
    after_accuracy=self.check_accuracy(node.node_D,parent_leaf_value)

    if now_accuracy+0.1 >after_accuracy:
        return False
    else:
        return True
  def del_node(self,node):
    # 删除子节点，并未释放空间，remove只会删除第一个元素，但这并不会造成问题，因为子节点必然互不相同
    return node.parent.children.remove(node)
  def bhd_tree(self,now_node):

    # 如果已经是根节点并且没有子节点，退出
    if now_node == self.root and now_node.children == []:

      return True
    # 如果是叶子节点，则直接考虑是否剪枝
    if now_node.leaf_value != None:

      if self.is_del(now_node):

        self.del_node(now_node)
     
      return True
    # 下面是针对中间节点的递归
    
    for child in reversed(now_node.children):
        self.bhd_tree(child)
    # 如果中间节点的所有子节点都没了，那给该中间节点赋予leaf_value,并向上追溯
    if now_node.children==[]:
        last_column_values = [sublist[-1] for sublist in now_node.node_D]   
        counter = Counter(last_column_values)   
        leaf_value,_ = counter.most_common(1)[0] 
        now_node.leaf_value=leaf_value
        self.bhd_tree(now_node)

  # # 剪枝
  # def check_accuracy(self,D,value):
  #   # 计算预测值与真实值的差异
  #   accuracy = sum(1 for d in D if d[-1] == value) / len(D)
  #   return accuracy
  # def is_del(self,node):
  #   now_accuracy=self.check_accuracy(node.node_D,node.leaf_value)
  #   # 查看父节点多数原则
  #   last_column_values = [sublist[-1] for sublist in node.parent.node_D]   
  #   counter = Counter(last_column_values)   
  #   parent_leaf_value,_ = counter.most_common(1)[0] 
  #   # 剪枝后准确率
  #   after_accuracy=self.check_accuracy(node.node_D,parent_leaf_value)
    
  #   if now_accuracy >after_accuracy:
  #       return False
  #   else:
  #       return True
  # def del_node(self,node):
  #   # 删除子节点，并未释放空间，remove只会删除第一个元素，但这并不会造成问题，因为子节点必然互不相同
  #   return node.parent.children.remove(node)
  # def bhd_tree(self,now_node):

  #   # 如果已经是根节点并且没有子节点，退出
  #   if now_node == self.root and now_node.children == []:
      

  #     return True
  #   # 如果是叶子节点，则直接考虑是否剪枝
  #   if now_node.leaf_value != None:
        
  #       if self.is_del(now_node):
          
  #         self.del_node(now_node)
        
  #       return True
  #   # 下面是针对中间节点的递归
    
  #   for child in now_node.children:
  #       self.bhd_tree(child)
  #   # 如果中间节点的所有子节点都没了，那向上追溯
  #   if now_node.children==[]:
  #       self.bhd_tree(now_node)

class predicts:
    def __init__(self,tree,X,y):
        self.tree = tree
        self.X = X
        self.y = y
        self.predict_result=[]
        # self.predict(self.X,tree.root)
    # classify=classifyer(tree=tree_gen,X=[['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑'],['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']],y=['是','否'])
    # ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑']
    # 1
    # p匹配上了 蜷缩
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 1
    # p匹配上了 稍蜷
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 0
    # p匹配上了 乌黑
    # ['是', '否']
     
    def predict(self,now_x,now_node):
       
        for x in now_x:
            # 如果是叶子节点，返回叶节点的取值。其实这个判断仅在根节点是叶子节点的特殊情况才会用到。如果考虑删除下面那句，又必然会多套一层函数，所以差别不是很大
            if now_node.leaf_value!=None:
                self.predict_result.append(int(now_node.leaf_value))
           
            feature_index=now_node.feature_index #获取本节点特征选取的index
            
            for child in now_node.children:
                # 仅进入对应取值的分支
                if math.isclose(x[feature_index],float(child.feature_val)):
                    
                    # 如果孩子是叶节点，返回叶节点的取值
                    if child.leaf_value != None:
                        self.predict_result.append(int(child.leaf_value))
                    else:
                        
                        self.predict([x],child)
                    break
                  # 1111111111
                else:
                  if child==now_node.children[-1]:
                    
                    # 提取每个子列表的最后一个元素  
                    last_column_values = [sublist[-1] for sublist in now_node.node_D]  
                      
                    # 使用 Counter 统计频次  
                    counter = Counter(last_column_values)  
                      
                    # 找到频次最高的元素及其频次  
                    most_common_value, most_common_count = counter.most_common(1)[0] 
                    
                    self.predict_result.append(int(most_common_value))
        
        return self.predict_result



        


In [ ]:

class predicts:
    def __init__(self,tree,X,y):
        self.tree = tree
        self.X = X
        self.y = y
        self.predict_result=[]
        # self.predict(self.X,tree.root)
    # classify=classifyer(tree=tree_gen,X=[['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑'],['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']],y=['是','否'])
    # ['青绿', '蜷缩', '浊响', '清晰', '凹陷', '硬滑']
    # 1
    # p匹配上了 蜷缩
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 1
    # p匹配上了 稍蜷
    # ['乌黑', '稍蜷', '沉闷', '稍糊', '稍凹', '软粘']
    # 0
    # p匹配上了 乌黑
    # ['是', '否']
     
    def predict(self,now_x,now_node):

        for x in now_x:
            # 如果是叶子节点，返回叶节点的取值。其实这个判断仅在根节点是叶子节点的特殊情况才会用到。如果考虑删除下面那句，又必然会多套一层函数，所以差别不是很大
            if now_node.leaf_value!=None:
                self.predict_result.append(int(now_node.leaf_value))

            feature_index=now_node.feature_index #获取本节点特征选取的index

            for child in now_node.children:
                # 仅进入对应取值的分支
                if math.isclose(x[feature_index],float(child.feature_val)):

                    # 如果孩子是叶节点，返回叶节点的取值
                    if child.leaf_value != None:
                        self.predict_result.append(int(child.leaf_value))
                    else:
                        
                        self.predict([x],child)
                    break
                  # 1111111111
                else:
                  if child==now_node.children[-1]:
                    
                    # 提取每个子列表的最后一个元素  
                    last_column_values = [sublist[-1] for sublist in now_node.node_D]  
                      
                    # 使用 Counter 统计频次  
                    counter = Counter(last_column_values)  
                      
                    # 找到频次最高的元素及其频次  
                    most_common_value, most_common_count = counter.most_common(1)[0] 
 
                    self.predict_result.append(int(most_common_value))
        
        return self.predict_result



        


下面是对函数的测试

In [1]:
def calculate_pk(D):
      A=[]

      for i in range(len(D)): 
        # 提取最后一列数据
        A.append(D[i][-1])
         
      # 创建一个字典来存储每个值（转换为字符串后）出现的次数  
      count_dict = {}  
        
      # 统计每个值（转换为字符串后）出现的次数  
      for item in A:  
          # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
          # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
          item_str = str(item)  
          if item_str in count_dict:  
              count_dict[item_str] += 1  
          else:  
              count_dict[item_str] = 1  
        
      # 计算列表中元素的总数  
      total_count = len(A)  
        
      # 计算每个值（转换为字符串后）出现的次数占总次数的比例  
      proportion_dict = {item_str: count / total_count for item_str, count in count_dict.items()} 
        
      # 打印结果  
      return proportion_dict


In [2]:
# calculate_pk(['是','是','是','是','是','是','是','是','否','否','否','否','否','否','否','否','否'])
calculate_pk([[1,1],[2,2],[2,3]])


{'1': 0.3333333333333333, '2': 0.3333333333333333, '3': 0.3333333333333333}

In [3]:
calculate_pk([[1,1],[2,2],[2,2]])

{'1': 0.3333333333333333, '2': 0.6666666666666666}

In [2]:
import math
def calculate_Entropy(A):
      p = calculate_pk(A)
      entropy = 0
      for i in p.values():
        entropy += i*math.log(i,2)
      return -entropy

In [10]:
print(calculate_Entropy(['是','是','是','是','是','是','是','是','否','否','否','否','否','否','否','否','否']))

0.9975025463691153


In [9]:
def calculate_Gain(self,D,a):
        if D==[]:
          print('D is empty')
          return 0
        if a<0 or a>=len(D[0]):
          print('a is not in D')
          return 0
        
        Ent_D = calculate_Entropy(D)
        # print(Ent_D)
        Ent_Di={}
        count_dict = {}  # 记录指定属性的取值的次数
        attribute_name=[]   # 记录指定属性的取值
        
       # 统计每个值（转换为字符串后）出现的次数  
        for item in D:  
        # 将项转换为字符串（这里简单地将所有非字符串类型转换为字符串）  
        # 注意：这可能会导致类型信息的丢失，例如1和1.0会被视为相同的字符串'1'  
          item_str = str(item[a])  
          if item_str in count_dict:  
            count_dict[item_str] += 1  
            Ent_Di[item_str].append(item)
          else:  
            count_dict[item_str] = 1
            Ent_Di[item_str]=[]
            Ent_Di[item_str].append(item)
            attribute_name.append(item_str)       

        # 计算列表中元素的总数
        total_count = len(D)
        # 计算每个值（转换为字符串后）出现的次数占总次数的比例
        attribute_weight = [count / total_count for count in count_dict.values()]
        # print(attribute_weight)
        # print(attribute_name)
        gain=Ent_D
        for i in range(len(attribute_name)):
          gain-=attribute_weight[i]*calculate_Entropy(Ent_Di[attribute_name[i]])
        print(gain)
        return gain,attribute_name

In [4]:
calculate_Gain(0,[[1,2,3,0],[2,3,1,0],[3,1,2,0],[1,2,3,0],[2,3,1,0],[3,1,2,1]],1)

0.6500224216483541
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
['2', '3', '1']
0.3166890883150208


(0.3166890883150208, ['2', '3', '1'])

In [10]:
# choose_A(0,[[2,2,3,0],[2,2,1,0],[2,2,2,0],[1,2,3,0],[3,3,3,0],[3,1,2,1]],[0,1,2])
def choose_A(self,D,A):
      max_gain=0
      for i in range(len(A)):
        gain,attribute_name=calculate_Gain(0,D,i)
        if gain>max_gain:
          max_gain=gain
          max_attribute_index=i
          max_attribute_name=attribute_name
      return max_attribute_index,max_attribute_name

In [11]:
a,b=choose_A(0,[[2,2,3,0],[2,2,1,0],[2,2,2,0],[1,2,3,0],[3,3,3,0],[3,1,2,1]],[0,1,2])

0.3166890883150208
0.6500224216483541
0.3166890883150208


In [12]:
a

1

In [13]:
b

['2', '3', '1']

任取上面三个版本之一运行，再运行下面这个cell即可查看到树的样子了，具体去看print_tree函数。

In [2]:
D = [  
    # 样本1: 色泽=1, 根蒂=2, 敲声=1, 纹理=1, 脐部=1, 触感=1, 12好瓜=1  
    [1, 2, 1, 1, 1, 1, 1],  
    # 样本2: 色泽=3, 根蒂=3, 敲声=2, 纹理=2, 脐部=3, 触感=2, 12好瓜=2  
    [3, 3, 2, 2, 3, 2, 2],  
    # 样本3: 色泽=3, 根蒂=2, 敲声=2, 纹理=1, 脐部=1, 触感=1, 12好瓜=1  
    [3, 2, 2, 1, 1, 1, 1],  
    # 样本4: 色泽=2, 根蒂=1, 敲声=3, 纹理=3, 脐部=2, 触感=1, 12好瓜=2  
    [2, 1, 3, 3, 2, 1, 2],  
    # 样本5: 色泽=1, 根蒂=3, 敲声=1, 纹理=1, 脐部=3, 触感=1, 12好瓜=1  
    [1, 3, 1, 1, 3, 1, 1],  
    # 样本6: 色泽=3, 根蒂=1, 敲声=1, 纹理=1, 脐部=2, 触感=2, 12好瓜=2  
    [3, 1, 1, 1, 2, 2, 2],  
    # 样本7: 色泽=2, 根蒂=2, 敲声=2, 纹理=1, 脐部=1, 触感=1, 12好瓜=1  
    [2, 2, 2, 1, 1, 1, 1],  
    # 样本8: 色泽=1, 根蒂=1, 敲声=3, 纹理=2, 脐部=2, 触感=2, 12好瓜=2  
    [1, 1, 3, 2, 2, 2, 2]  
]
# # 正常构造
A = [0,1,2,3,4,5]
t=TreeGenerate(D,A)
# print("剪枝前：")
print_tree(t.root)
# t.bhd_tree(t.root)
# print("剪枝后：")
# print_tree(t.root)

Node_D_val: 4, Feature Index: 4, node_node_D: [[1, 2, 1, 1, 1, 1, 1], [3, 3, 2, 2, 3, 2, 2], [3, 2, 2, 1, 1, 1, 1], [2, 1, 3, 3, 2, 1, 2], [1, 3, 1, 1, 3, 1, 1], [3, 1, 1, 1, 2, 2, 2], [2, 2, 2, 1, 1, 1, 1], [1, 1, 3, 2, 2, 2, 2]], node_children: [<__main__.Node object at 0x00000207E7DBFDD8>, <__main__.Node object at 0x00000207E7DBF898>, <__main__.Node object at 0x00000207E7DBFD30>], Leaf Value: None
    Node_D_val: 1, Feature Index: None, node_node_D: [[1, 2, 1, 1, 1, 1, 1], [3, 2, 2, 1, 1, 1, 1], [2, 2, 2, 1, 1, 1, 1]], node_children: [], Leaf Value: 1
    Node_D_val: 3, Feature Index: 3, node_node_D: [[3, 3, 2, 2, 3, 2, 2], [1, 3, 1, 1, 3, 1, 1]], node_children: [<__main__.Node object at 0x00000207E7DBF518>, <__main__.Node object at 0x00000207E7DBF710>], Leaf Value: None
        Node_D_val: 2, Feature Index: None, node_node_D: [[3, 3, 2, 2, 3, 2, 2]], node_children: [], Leaf Value: 2
        Node_D_val: 1, Feature Index: None, node_node_D: [[1, 3, 1, 1, 3, 1, 1]], node_children: [],

0